In [1]:
import GPRutils
import DESutils

import numpy as np
import astropy.table as tb
import astropy.units as u

import warnings
warnings.filterwarnings("ignore")

exps = DESutils.findExpNums()
gband = DESutils.bandDict["g"]
rband = DESutils.bandDict["r"]
iband = DESutils.bandDict["i"]
zband = DESutils.bandDict["z"]

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [17]:
for expNum in zband[:5] + gband[:5]:
    dataContainers = {
        "RCmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/NelderMead_max/GPR.{expNum}.?RC.fits"),
        "Rmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/NelderMead_max/GPR.{expNum}.?R.fits"),
        
        "RC": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/NelderMead/GPR.{expNum}.?RC.fits"),
        "R": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/NelderMead/GPR.{expNum}.?R.fits"),
                
        "RCminmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/NelderMead_minmax/GPR.{expNum}.?RC.fits"),
        "Rminmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/NelderMead_minmax/GPR.{expNum}.?R.fits"),

        "RCmin": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/NelderMead_min/GPR.{expNum}.?RC.fits"),        
        "Rmin": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/NelderMead_min/GPR.{expNum}.?R.fits"),
        
        
#         "altR": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/L_BFGS_B/GPR.{expNum}.?R.fits"),
#         "altRmin": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/L_BFGS_B_min/GPR.{expNum}.?R.fits"),
#         "altRmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/L_BFGS_B_max/GPR.{expNum}.?R.fits"),
#         "altRminmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/L_BFGS_B_minmax/GPR.{expNum}.?R.fits"),

#         "altRC": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/L_BFGS_B/GPR.{expNum}.?RC.fits"),
#         "altRCmin": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/L_BFGS_B_min/GPR.{expNum}.?RC.fits"),        
#         "altRCmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/L_BFGS_B_max/GPR.{expNum}.?RC.fits"),
#         "altRCminmax": GPRutils.loadFITS(f"/home/fortino/GPRSolutions/L_BFGS_B_minmax/GPR.{expNum}.?RC.fits"),
    }

    print(f"{expNum}, {list(dataContainers.values())[0].band}")
    names = ["Method", "xi0", "xif", "Reduction", "Stars/arcmin^2", "K Variance", "Outer Scale", "Diameter", "Wind X", "Wind Y"]
    header = "".join([f"{name:<15}" for name in names])
    print(header)
    
    for i, (method, dC) in enumerate(dataContainers.items()):
        vals = [method]
        if not (i & 2 - 1) and i != 0:
            print()
        
        xi0 = dC.header["xi0"]
        Xerr = dC.header["xi0_Xerr"]
        Yerr = dC.header["xi0_Yerr"]
        xi0err = np.sqrt(Xerr**2 + Yerr**2)
        vals.append(f"{xi0:.2f} ± {xi0err:.2f}")

        xif = dC.header["xif"]
        Xerr = dC.header["xif_Xerr"]
        Yerr = dC.header["xif_Yerr"]
        xiferr = np.sqrt(Xerr**2 + Yerr**2)
        vals.append(f"{xif:.2f} ± {xiferr:.2f}")

        red = xi0/xif
        rederr = np.sqrt(((xi0err/xi0)**2 + (xiferr/xif)**2) * red**2)
        vals.append(f"{red:.2f} ± {rederr:.2f}")
        
        N = len(dC.TV[dC.TV["Maskf"]])
        starDensity = np.round((N / (3*u.deg**2)).to(u.arcmin**-2), 3)
        vals.append(starDensity.value)
        
        vals.extend(np.abs(np.around(dC.params, 7)))

        line = "".join([f"{param:<15}" for param in vals])
        print(line)
    print()
    print()
    
# a = b/c
# (da/a)^2 = (db/b)^2 + (dc/c)^2

348819, z
Method         xi0            xif            Reduction      Stars/arcmin^2 K Variance     Outer Scale    Diameter       Wind X         Wind Y         
RCmax          82.98 ± 1.72   4.99 ± 0.50    16.63 ± 1.71   0.914          324.3305769    1.533597       0.0287053      0.0171639      0.0306427      
Rmax           82.82 ± 1.72   5.86 ± 0.52    14.14 ± 1.30   0.914          429.2719121    1.6826592      0.0134032      0.0016738      0.0733438      

RC             83.08 ± 1.69   5.64 ± 0.49    14.73 ± 1.30   0.916          220.7504306    0.5817545      0.0084507      0.0122671      0.030748       
R              83.03 ± 1.69   6.92 ± 0.51    12.01 ± 0.93   0.915          394.0468822    0.6892833      0.0164337      0.0033909      0.0150839      

RCminmax       84.65 ± 2.69   8.07 ± 0.97    10.49 ± 1.31   0.641          162.8510867    0.4765492      0.0266703      0.0278908      0.0591325      
Rminmax        84.19 ± 2.69   8.56 ± 1.02    9.84 ± 1.22    0.641          308.915